In [20]:
import pandas as pd
import numpy as np
import collections

# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split 

# инструмент для создания и обучения модели 
from sklearn.ensemble import RandomForestRegressor 

# инструменты для оценки точности модели 
from sklearn import metrics 

# модуль для работы с датами.
from datetime import datetime, timedelta
import time

# модуль для работы с регулярными выражениями
import re

# модуль для работы с полиноминальными признаками
from sklearn.preprocessing import PolynomialFeatures

# Модули для визуализации
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Модули для нахождения выбросов
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [21]:
# Импортируем датасет и сразу меняем названия для удобства.
df = pd.read_csv('main_task_new.csv')
df.rename(columns={'Restaurant_id':'restaurant_id', 'City':'city', 'Cuisine Style':'cuisine_style', 
                   'Ranking':'ranking', 'Rating':'rating', 'Rating':'rating', 'Price Range':'price_range', 
                   'Number of Reviews':'number_of_reviews', 'Reviews':'reviews', 
                   'URL_TA':'url_ta', 'ID_TA':'id_ta'}, inplace=True)


# Для удобства при тестировании разобьем код на логические блоки и обернем в функции.

In [22]:
# Функция форматирует столбец cuisine_style
def form_cuisine(a):
    a = str(a)
    a = a.replace("'",'')
    a = a.replace('[', '')
    a = a.replace(']', '')
    a = a.split(', ')
    return a

# Функция для поиска самой встречающийся кухни.
def top_cuisine(a):
    cnt = collections.Counter()
    for i in a:
        for j in i:
            j = str(j)
            cnt[j] += 1
    return cnt.most_common(1)

# Функция для подсчета количества рпзличных кухонь.
def quant_cuisine(a):
    cnt = collections.Counter()
    for i in a:
        for j in i:
            j = str(j)
            cnt[j] += 1
    return cnt 

# Функции для отделения даты от коментариев
def form_reviews(a):
    a = str(a)
    a = a.strip('[]').split('[')
    del a[0]
    return a

# Функция для создания столбца разницы в днях коментвриев
def time_review(a):
    if len(a) == 2:
        a = a[0] - a[1]
    else: a = np.nan
    return abs(a)


# Функция для замены пустых списков в reviews_date на NaN
def NaN_for_rev(a):
    if a == []:
        a = np.nan
    return a

# Функция для замены знаков дол. числовыми значениями
def price_range_form(a):
    if a == '$$ - $$$':
        a = 999
    elif a == '$$$$':
        a = 9999
    elif a == '$':
        a = 9
    return a

# Функция для подсчета количества разнообразных кухонь
def len_cuisine(a):
    a = len(a)
    return a

# Функция для подсчета количества символов в коментариях.

def form_reviews(a):
    a = str(a)
    a = a.split('], [')
    a[0] = a[0].replace("'",'')
    a[0] = a[0].replace('[', '')
    a[0] = a[0].replace(']', '')
    a[0] = a[0].replace(',','')
    a[0] = a[0].replace(' ','')
    del a[1]
    a = a[0]
    
    return len(a)

# Функция для создания DUMMY признаков из столбца cuisine_style
def create_dummies(df, col, lst):
    for elem in lst:
        df[elem] = df[col].apply(lambda x: 1 if elem in x else 0)
    return df

# Функции для определения выбросов
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]


def outliers_z_score(ys, threshold=3):
    mean_y = np.mean(ys)
    std_y = np.std(ys)
    z_scores = [(y - mean_y) / std_y for y in ys]
    return np.where(np.abs(z_scores) > threshold)[0]


# Фунция для разделения столбца со временем
def time_column(a):
    if a == 0:
        a = a
    elif len(a) == 2:
        a = a[1]
    return a


# Функция для построения графиков распределения
def diagnostic_plots(df, variable, title):
    fig, ax = plt.subplots(figsize=(10,7))
    # гистограмма
    plt.subplot(2, 2, 1)
    df[variable].hist(bins=30)
    ## Q-Q plot
    plt.subplot(2, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    # ящик с усами
    plt.subplot(2, 2, 3)
    sns.violinplot(x=df[variable])    
    # ящик с усами
    plt.subplot(2, 2, 4)
    sns.boxplot(x=df[variable])  
    fig.suptitle(title)
    plt.show()


In [23]:
def read_df(df):
    # Импортируем датасет и сразу меняем названия для удобства.
    df = pd.read_csv('main_task_new.csv')
    df.rename(columns={'Restaurant_id':'restaurant_id', 'City':'city', 'Cuisine Style':'cuisine_style', 
                   'Ranking':'ranking', 'Rating':'rating', 'Rating':'rating', 'Price Range':'price_range', 
                   'Number of Reviews':'number_of_reviews', 'Reviews':'reviews', 
                   'URL_TA':'url_ta', 'ID_TA':'id_ta'}, inplace=True)
    return df



def preproc_data(df):
    # Создаем столбец с датами
    pattern=re.compile('\d+[/]\d+[/]\d+')
    df["reviews_date"] = df.reviews.apply(lambda x:re.findall(pattern,x))
    df["reviews_date"] = df["reviews_date"].apply(lambda y:[pd.to_datetime(item)for item in y])
    
    
    # Создадим признак разници времени написания коментариев
    # и приведем данные к числовомк типу.
    df['reviews_dif_date'] = df.reviews_date.apply(time_review)
    df['reviews_dif_date'] = df['reviews_dif_date'].dt.days
    df['reviews_date'] = df.reviews_date.apply(NaN_for_rev)
    
    
    # Добавим признак самой популярной кухни в городе
    top_cuisine = {'Paris': "['French']", 
          'Stockholm' : "['European', 'Swedish']", 'London': "['Bar', 'British', 'Pub']", 
          'Berlin': "['Italian']", 'Munich': "['Italian']" , 
          'Oporto' : "['European', 'Portuguese']", 'Milan': "['Italian']", 'Bratislava': "['Pub']", 
          'Vienna': "['Austrian', 'European']", 'Rome': "['Italian']",    
          'Barcelona': "['Spanish']", 'Madrid': "['Spanish']", 'Dublin': "['Irish', 'Bar', 'Pub']", 
          'Brussels': "['Belgian', 'European']", 'Zurich': "['Swiss', 'European']",  
          'Warsaw': "['Polish', 'European']", 'Budapest': "['European', 'Hungarian']", 
          'Copenhagen': "['European', 'Danish']", 'Amsterdam': "['Dutch', 'European']", 
          'Lyon': "['French']", 'Hamburg': "['Italian']", 'Lisbon': "['European', 'Portuguese']", 
          'Prague': "['European', 'Czech']", 'Oslo': "['Pub']", 
          'Helsinki': "['Bar', 'Pub']", 'Edinburgh': "['Cafe']",'Ljubljana': "['European', 'Slovenian']", 
          'Athens': "['Greek']", 'Luxembourg': "['French']", 'Krakow': "['Polish', 'European']", 
          'Geneva': "['French', 'Mediterranean', 'European']" }
    df['top_cuisine'] = df['city'].map(top_cuisine)
    
    
    # Создадим признак населения городов
    population = {'Paris': 2196936, 'Stockholm' : 925934, 'London': 8416999, 'Berlin': 3469849 , 'Munich': 1488202 , 
              'Oporto' : 214349, 'Milan': 1378689, 'Bratislava': 437725, 'Vienna': 1897491, 'Rome': 2870500, 
              'Barcelona': 1664182, 'Madrid': 3266126, 'Dublin': 1173179, 'Brussels': 185103, 'Zurich': 42873,
              'Warsaw': 179065,  'Budapest': 1752286, 'Copenhagen': 615993, 'Amsterdam': 872757, 'Lyon': 506615,
              'Hamburg': 1841179, 'Lisbon': 505526, 'Prague': 1335084, 'Oslo': 673469, 'Helsinki': 656611, 'Edinburgh': 488100,
              'Ljubljana': 284355, 'Athens': 664046, 'Luxembourg': 114303, 'Krakow': 779115, 'Geneva': 200548 }

    df['population'] = df['city'].map(population)
    
    
    # Создадим признак площади городов
    square = {'Paris': 105.4, 'Stockholm' : 188, 'London': 1572, 'Berlin': 891.68 , 'Munich': 310.71 , 
              'Oporto' : 41.66, 'Milan': 181.67, 'Bratislava': 368, 'Vienna': 414.75, 'Rome': 1287.36, 
              'Barcelona': 101.3, 'Madrid': 607, 'Dublin': 318, 'Brussels': 32.61, 'Zurich': 91.88,
              'Warsaw': 517.2,  'Budapest': 525.14, 'Copenhagen': 86.40, 'Amsterdam': 219.4, 'Lyon': 47.87,
              'Hamburg': 755.09, 'Lisbon': 100.05, 'Prague': 496, 'Oslo': 454, 'Helsinki': 213.8, 'Edinburgh': 118,
              'Ljubljana': 163.8, 'Athens': 412, 'Luxembourg': 51.46, 'Krakow': 327, 'Geneva': 15.93 }

    df['square'] = df['city'].map(square)
    
    
    # Выделим значения NAN в отдельный признак
    df['cuisine_style_is_NAN'] = pd.isna(df['cuisine_style']).astype('uint8')

    # Заполним пропуски в столбце cuisine_style на самую часто встречающуюся Vegetarian Friendly 
    df.cuisine_style.fillna('Vegetarian Friendly', inplace = True)
    
    # Используем функцию form_cuisine для форматирования столбца cuisine_style
    df['cuisine_style'] = df.cuisine_style.apply(form_cuisine)
       
    # Создадим признак сетевого ресторана где 1 = сетевой 0 = не сетевой.
    rest_chain_temp = df.restaurant_id.value_counts()
    rest_chain_temp = rest_chain_temp[rest_chain_temp >2]
    rest_chain = []
    for i in df.restaurant_id:
        if i in rest_chain_temp:
            rest_chain.append(1)
        elif i not in rest_chain_temp:
            rest_chain.append(0)
    rest_chain = pd.Series(rest_chain)
    df['rest_chain'] = rest_chain
    
    # Заменяем знаки дол. числовыми значениями.
    df['price_range'] = df['price_range'].apply(price_range_form)
    
    # Создадим признак с количеством разнообразных кухонь
    df['quantity_cuisine'] = df['cuisine_style'].apply(len_cuisine)
    
    # Создадим признак с количеством символов в коментариях.
    df['quantity_words'] = df.reviews.apply(form_reviews)
    
    # Создадим признак количество ресторвнов в городе
    df['rest_in_city']=df.groupby(["city"])['restaurant_id'].transform('count')
    
    # Разделим reviews_date на 2 признака.
    df['reviews_date'] = df['reviews_date'].fillna(0)
    df['reviews_date_1'] = df['reviews_date'].apply(time_column)
    df['reviews_date_2'] = df['reviews_date'].apply(time_column)
    
    # Ячейка для выделения NaN признаков
    # Отделяем пропуски в отдельный признак
    df['number_of_reviews_is_NAN'] = pd.isna(df['number_of_reviews']).astype('uint8')
    df['price_range_is_NAN'] = pd.isna(df['price_range']).astype('uint8')
              
    # приводим к числовому виду столбец id_ta
    df['id_ta'] = df['id_ta'].apply(lambda x: int(x[1::]))
    
    return df



def full_NA(df):
    # Найдем самый часто встречающийся ценовой диапазон по городам и выясним, что это везде 999.
    # Заполним пропуски этим значением 
    a = df.groupby('city')['price_range'].value_counts().reset_index(name='counts')
    city_top_price = a.iloc[0:93:3]
    df.price_range = df.price_range.fillna(999)
    
    df['number_of_reviews'].fillna(0, inplace = True)
    df['reviews_dif_date'].fillna(0, inplace = True)
    
    return df


def Polynom(df):
    # Будем эксперементировать попробуем умножать делить 
    # выбраные признаки и посмотрим как это будет сказываться на МАЕ
    df['rest_in_city/population'] = df['rest_in_city'] / df['population']
    df['rest_in_city/population'] = df['rest_in_city'] / df['population'] 
    df['ranking*number_of_reviews'] = df['ranking']*df['number_of_reviews']
    df['ranking*quantity_cuisine'] = df['ranking']*df['quantity_cuisine']
    df['ranking/population'] = df['ranking'] / df['population'] 
    df['ranking/rest_in_city'] = df['ranking']/df['rest_in_city']
    df['number_of_reviews/ranking'] = df['number_of_reviews']/df['ranking']
    df['ranking/quantity_cuisine'] = df['ranking']/df['quantity_cuisine']
    
    
    # Возьмем средние значения признаков по городам
    df['ranking_city_mean'] = df['city'].map(df.groupby('city')['ranking'].mean())
    df['reviews_city_mean'] = df['city'].map(df.groupby('city')['number_of_reviews'].mean())
    df['quantity_cuisine_mean'] = df['city'].map(df.groupby('city')['quantity_cuisine'].mean())
    df['ranking/city'] = df['city'].map(df.groupby('city')['ranking'].mean())
    df['ranking_city_median'] = df['city'].map(df.groupby('city')['ranking'].median())
    df['reviews_city_median'] = df['city'].map(df.groupby('city')['number_of_reviews'].median())
    # Логарифмируем признак ranking
    df['ranking'] = np.log(df['ranking'])
    
    # Пробуем нормализовать распределение ранкинга по городам.
    df["ranking_norm"] = df.groupby("city").ranking.apply(lambda x: (x - x.min())/(x.max() - x.min()))
    
    return df


def dummy(df):
    # Создадим Dummy признаки для city и cuisine_style.
    X_dummy = df.copy()
    cuisine_list = list(pd.Series(quant_cuisine(X_dummy.cuisine_style)).index)
    X_dummy_price_range = pd.get_dummies(X_dummy.price_range, prefix = "DMY", prefix_sep = '-')
    
    X_dummy_city = pd.get_dummies(X_dummy.city, prefix = "DMY", prefix_sep = '-')
    X_dummy_cuisine = create_dummies(X_dummy, 'cuisine_style', cuisine_list)
    X_dummy_top_cuisine = pd.get_dummies(X_dummy.top_cuisine, prefix = "DMY", prefix_sep = '-')
    
    X = pd.concat([X_dummy_cuisine, X_dummy_city, X_dummy_top_cuisine,X_dummy_price_range], axis=1)
        
    return X

def X(X):
    # Попробуем добавить еще полиномиальных признаков. После множественных тестов были оставлены только улудшающие модель. 
    pf = PolynomialFeatures(2)
    poly_features = pf.fit_transform(X[['number_of_reviews','quantity_cuisine',
                                        'rest_in_city','square','rest_in_city',
                                        'ranking','quantity_cuisine_mean',
                                        'id_ta','reviews_dif_date','rest_in_city'
                                       ]])
    
    
    poly_df = pd.DataFrame(poly_features)
    X = pd.concat([X, poly_df], axis=1)
    
    # Удаляем столбцы с номинативными признаками и признаками которые ухудшают модель.
    X = X.drop([ 'city', 'cuisine_style',
       'price_range', 'reviews', 'url_ta',
       'reviews_date', 'top_cuisine','reviews_date_1','reviews_date_2',
       'quantity_words','price_range',
       ], axis = 1)
    
    return X



def start_ML(X):
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
    y = X['rating']
    X = X.drop(['restaurant_id', 'rating'], axis = 1)
    
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
    # Для тестирования мы будем использовать 25% от исходного датасета.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    
    # Создаём модель  
    regr = RandomForestRegressor(n_estimators=100, random_state=42)  
    
    # Обучаем модель на тестовом наборе данных  
    regr.fit(X_train, y_train)  
          
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
    y_pred = regr.predict(X_test)  
    
    
    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
    # Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.

    return print('MAE:', metrics.mean_absolute_error(y_test, y_pred))



In [24]:
df = preproc_data(df)
df = full_NA(df)
df = Polynom(df)
df = dummy(df)
df = X(df)
df = start_ML(df)

